In [5]:
# Imports

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
# Students

students = pd.read_csv('input/students.csv')
students

In [25]:
katas = pd.read_csv('input/katas.csv')
katas.head()

,kata,date,minutes
0,RegExp Fun #1 - When I miss few days of gym,NaN,NaN
1,Deodorant Evaporator,NaN,NaN
2,Arithmetic List!,NaN,NaN
3,Fake Binary,NaN,NaN
4,"If you can't sleep, just count sheep!!",NaN,NaN


In [6]:
# https://www.codewars.com/api/v1/users/Livia Canet/code-challenges/completed

def get_completed(user, 
                  host='https://www.codewars.com/api/v1/users/', 
                  path='/code-challenges/completed'):
    url = host + user + path
    resp = requests.get(url)
    return resp.json()
 

g = get_completed(ALUMNOS['Paula Postigo Rodrigo'])

In [8]:
g['data']

[{'id': '5720a81309e1f9b232001c5b',
  'name': 'RegExp Fun #1 - When I miss few days of gym',
  'slug': 'regexp-fun-number-1-when-i-miss-few-days-of-gym',
  'completedLanguages': ['python'],
  'completedAt': '2019-02-01T16:20:21.241Z'},
 {'id': '5506b230a11c0aeab3000c1f',
  'name': 'Deodorant Evaporator',
  'slug': 'deodorant-evaporator',
  'completedLanguages': ['python'],
  'completedAt': '2019-01-31T10:23:30.056Z'},
 {'id': '541da001259d9ca85d000688',
  'name': 'Arithmetic List!',
  'slug': 'arithmetic-list',
  'completedLanguages': ['python'],
  'completedAt': '2019-01-25T00:38:39.165Z'},
 {'id': '57eae65a4321032ce000002d',
  'name': 'Fake Binary',
  'slug': 'fake-binary',
  'completedLanguages': ['python'],
  'completedAt': '2019-01-24T22:47:31.068Z'},
 {'id': '5b077ebdaf15be5c7f000077',
  'name': "If you can't sleep, just count sheep!!",
  'slug': 'if-you-cant-sleep-just-count-sheep',
  'completedLanguages': ['python'],
  'completedAt': '2019-01-24T08:44:25.357Z'},
 {'id': '50654d

In [ ]:
'''
curl "http://www.codewars.com/api/v1/users/some_user/code-challenges/completed?page=0"

{
  "totalPages": 1,
  "totalItems": 1, 
  "data": [{
          "id":"514b92a657cdc65150000006",
          "name":"Multiples of 3 and 5",
          "slug":"multiples-of-3-and-5",
          "completedAt": "2017-04-06T16:32:09Z",
          "completedLanguages":[ 
            "javascript",
            "coffeescript",
            "ruby",
            "javascript",
            "ruby",
            "javascript",
            "ruby",
            "coffeescript",
            "javascript",
            "ruby",
            "coffeescript"
          ]
  }]
}

'''

'''
curl "https://www.codewars.com/api/v1/code-challenges/valid-braces"

{
    "id": "5277c8a221e209d3f6000b56",
    "name": "Valid Braces",
    "slug": "valid-braces",
    "category": "algorithms",
    "publishedAt": "2013-11-05T00:07:31Z",
    "approvedAt": "2013-12-20T14:53:06Z",
    "languages": [
        "javascript",
        "coffeescript"
    ],
    "url": "http://www.codewars.com/kata/valid-braces",
    "rank": {
        "id": -4,
        "name": "4 kyu",
        "color": "blue"
    },
    "createdBy": {
        "username": "xDranik",
        "url": "http://www.codewars.com/users/xDranik"
    },
    "approvedBy": {
        "username": "xDranik",
        "url": "http://www.codewars.com/users/xDranik"
    },
    "description": "Write a function called `validBraces` that takes a string of braces, and determines if the order of the braces is valid. `validBraces` should return true if the string is valid, and false if it's invalid.\n\nThis Kata is similar to the Valid Parentheses Kata, but introduces four new characters. Open and closed brackets, and open and closed curly braces. Thanks to @arnedag for the idea!\n\nAll input strings will be nonempty, and will only consist of open parentheses '(' , closed parentheses ')', open brackets '[', closed brackets ']', open curly braces '{' and closed curly braces '}'. \n\n<b>What is considered Valid?</b>\nA string of braces is considered valid if all braces are matched with the correct brace. <br/>\nFor example:<br/>\n'(){}[]' and '([{}])' would be considered valid, while '(}', '[(])', and '[({})](]' would be considered invalid.\n\n\n<b>Examples:</b> <br/>\n`validBraces( \"(){}[]\" )` => returns true <br/>\n`validBraces( \"(}\" )` => returns false <br/>\n`validBraces( \"[(])\" )` => returns false <br/>\n`validBraces( \"([{}])\" )` => returns true <br/>\n",
    "totalAttempts": 4911,
    "totalCompleted": 919,
    "totalStars": 12,
    "tags": [
        "Algorithms",
        "Validation",
        "Logic",
        "Utilities"
    ]
}

'''

In [ ]:
# pruebas de git